# Creating a P2PKH transaction

In this section we'll create a P2PKH transaction from scratch in python. We'll go through each part of the transaction, how it's constructed, signed, and we'll test it using the bitcoin core test framework.

## Prerequisite knowledge
- For all notebooks:
    - A high level understanding of the bitcoin. e.g. [Mastering Bitcoin](https://github.com/bitcoinbook/bitcoinbook) by Andreas Antonopoulos UTXO model, in particular [Chapter 6](https://github.com/bitcoinbook/bitcoinbook/blob/develop/ch06.asciidoc).
    - A conceptual understanding of [hash functions](https://www.thesslstore.com/blog/what-is-a-hash-function-in-cryptography-a-beginners-guide).
    - [Hexadecimal notation](https://inst.eecs.berkeley.edu/~cs61bl/r//cur/bits/decimal-binary-hex.html?topic=lab28.topic&step=2&course=) and [endianness](https://www.freecodecamp.org/news/what-is-endianness-big-endian-vs-little-endian/).


- Specific to this notebook:
    - SHA256, HASH256, HASH160 - '[Hash Functions chapter](https://github.com/DariusParvin/bitcoin-tx-tutorial/blob/main/appendix/hash-functions.ipynb)'
    - Base58 addresses - '[Addresses chapter](https://github.com/DariusParvin/bitcoin-tx-tutorial/blob/main/appendix/Addresses.ipynb)'
    - Bitcoin Script basics - '[Bitcoin Script chapter](https://github.com/DariusParvin/bitcoin-tx-tutorial/blob/main/appendix/Bitcoin%20Script.ipynb)'

## Setup 

### Requirements
For this exercise we'll need Bitcoin Core. This notebook has been tested with [v24.0.1](https://github.com/bitcoin/bitcoin/releases/tag/v24.0.1).

Below, set the paths for:
1. The bitcoin core functional test framework directory.
2. The directory containing bitcoin-tx-tutorial.

**You'll need to edit these next two lines for your local setup.**

In [1]:
path_to_bitcoin_functional_test = "/Users/masashi_mac_ssd/Developer/bitcoin/test/functional"
path_to_bitcoin_tx_tutorial = "/Users/masashi_mac_ssd/Developer/bitcoin-tx-tutorial"

### Setup bitcoin core test framework
Start up regtest mode, delete any regtest network history so we are starting from scratch. 

In [2]:
import sys

# Add the functional test framework to our PATH
sys.path.insert(0, path_to_bitcoin_functional_test)
from test_framework.test_shell import TestShell

# Add the bitcoin-tx-tutorial functions to our PATH
sys.path.insert(0, path_to_bitcoin_tx_tutorial)
from functions import *

import json

In [3]:
# Setup our regtest environment
test = TestShell().setup(
    num_nodes=1, 
    setup_clean_chain=True
)

node = test.nodes[0]

2025-12-02T17:44:35.211482Z TestFramework (INFO): PRNG seed is: 5808470798741060957
2025-12-02T17:44:35.213460Z TestFramework (INFO): Initializing test directory /var/folders/yk/mx8jn78j6xq0w5l4z9d8hbvw0000gn/T/bitcoin_func_test_1s1nxa5r


Create a wallet so that we can fund our first output using the `sendtoaddress` command. Mine 101 blocks so that the first block subsidy will have sufficient block maturity (100 blocks) to spend from.

In [4]:
# JSONRPCExceptionをインポート
from test_framework.authproxy import JSONRPCException

def setup_wallet(node, wallet_name='mywallet'):
    """ウォレットのセットアップと初期化"""
    try:
        node.createwallet(wallet_name=wallet_name)
    except JSONRPCException as e:
        if "Database already exists" in str(e):
            try:
                node.loadwallet(wallet_name)
            except JSONRPCException as e2:
                if "is already loaded" in str(e2):
                    pass  # 既にロードされている場合は無視
                else:
                    raise e2
        else:
            raise e
    return node.get_wallet_rpc(wallet_name)

def display_wallet_info(wallet, address):
    """ウォレット情報を表示"""
    # 基本情報を取得
    wallet_info = wallet.getwalletinfo()
    address_info = wallet.getaddressinfo(address)
    balance = wallet.getbalance()
    utxos = wallet.listunspent()
    total_utxo_value = sum(utxo['amount'] for utxo in utxos)
    
    # 情報を表示
    print("=== ウォレット情報 " + "=" * 30)
    print(f"・ウォレット名: {wallet_info['walletname']}")
    print(f"・バージョン: {wallet_info.get('walletversion', 'N/A')}")
    print(f"・ネットワーク: regtest")
    print(f"・現在の残高: {balance:,.8f} BTC")
    print(f"・UTXO数: {len(utxos)} ")
    
    print("\n" + "=== アドレス情報 " + "=" * 30)
    print(f"・アドレス: {address}")
    print(f"・公開鍵: {address_info.get('pubkey', 'N/A')}")
    print(f"・派生パス: {address_info.get('hdkeypath', 'N/A')}")
    
    # UTXOサマリー
    if utxos:
        print("\n" + "=== UTXOサマリー " + "=" * 30)
        for i, utxo in enumerate(utxos[:3], 1):
            print(f"  {i}. {utxo['amount']:>18.8f} BTC")
        if len(utxos) > 3:
            print(f"  ... 他 {len(utxos)-3} 件のUTXOがあります")
    

# メイン処理
def main():
    # ウォレットをセットアップ
    wallet = setup_wallet(node)
    address = wallet.getnewaddress()
    
    # 初期ブロック生成
    print("=== 初期ブロック生成中... ===")
    result = node.generatetoaddress(nblocks=101, address=address, called_by_framework=True)
    current_blockcount = node.getblockcount()
    print(f"✓ ブロック高 {current_blockcount} に達しました"+"\n")
    
    # ウォレット情報を表示
    display_wallet_info(wallet, address)

# メイン処理を実行
if __name__ == "__main__":
    main()

=== 初期ブロック生成中... ===
✓ ブロック高 101 に達しました

=== ウォレット情報 ==============================
・ウォレット名: mywallet
・バージョン: 169900
・ネットワーク: regtest
・現在の残高: 50.00000000 BTC
・UTXO数: 1 

=== アドレス情報 ==============================
・アドレス: bcrt1qt3lm8efae7g0d8nl6q8edf0yklxptpylv8l4mh
・公開鍵: 028f15290696e376494957267170456f2d654d69000fbdba99d18b8b4baf77fa58
・派生パス: m/84h/1h/0h/0/0

=== UTXOサマリー ==============================
  1.        50.00000000 BTC


### Create a P2PKH UTXO

In order to create a transaction spending from a P2PKH UTXO, we'll first need to create the UTXO that is locked with a p2pkh script. To do that, we'll create a P2PKH address from a private key, and fund it using the bitcoind wallet created in the setup step.

#### Create a p2pkh address 
For more on this step, review the 'Addresses' notebook.

In [5]:
sender_privkey = bytes.fromhex("1111111111111111111111111111111111111111111111111111111111111111")
print(f"秘密鍵 (hex): {sender_privkey.hex()}")
print(f"秘密鍵 (int): {int(sender_privkey.hex(), 16)}")

# 公開鍵を生成
sender_pubkey = privkey_to_pubkey(sender_privkey)
print(f"\n公開鍵 (圧縮形式): {sender_pubkey.hex()}")
print(f"公開鍵 (長さ): {len(sender_pubkey)} バイト")

# P2PKHアドレスを生成
sender_p2pkh_addr = pk_to_p2pkh(sender_pubkey, network="regtest")
print(f"\nP2PKHアドレス: {sender_p2pkh_addr}")

# アドレスの検証（オプション）
print(f"\nアドレスの検証:")
print(f"- 先頭文字: {sender_p2pkh_addr[0]}")
print(f"- 長さ: {len(sender_p2pkh_addr)} 文字")
print(f"- チェックサム: {sender_p2pkh_addr[-4:]}")


# mywalletのP2PKHアドレス生成
print("\n" + f"===mywalletのP2PKHアドレス生成===") 
try:
    # 公開鍵をバイト列に変換
    mywallet_pubkey_bytes = bytes.fromhex("02ac8c65108e5a81e4c565b57f59241b0b94bb19541f28323c3bec76f3011f1e02")
    print(f"公開鍵 (hex): {mywallet_pubkey_bytes.hex()}")
    print(f"公開鍵 (長さ): {len(mywallet_pubkey_bytes)} バイト")
    
    # P2PKHアドレスを生成（正しい変数名を使用）
    mywallet_p2pkh_addr = pk_to_p2pkh(mywallet_pubkey_bytes, network="regtest")
    print(f"\nP2PKHアドレス: {mywallet_p2pkh_addr}")
    
    # アドレスの検証
    print("\nアドレスの検証:")
    print(f"- 先頭文字: {mywallet_p2pkh_addr[0]}")
    print(f"- 長さ: {len(mywallet_p2pkh_addr)} 文字")
    print(f"- チェックサム: {mywallet_p2pkh_addr[-4:]}")
    
except Exception as e:
    print(f"エラーが発生しました: {e}")
    import traceback
    traceback.print_exc()  # 詳細なエラー情報を表示

秘密鍵 (hex): 1111111111111111111111111111111111111111111111111111111111111111
秘密鍵 (int): 7719472615821079694904732333912527190217998977709370935963838933860875309329

公開鍵 (圧縮形式): 034f355bdcb7cc0af728ef3cceb9615d90684bb5b2ca5f859ab0f0b704075871aa
公開鍵 (長さ): 33 バイト

P2PKHアドレス: n4XmX91N5FfccY678vaG1ELNtXh6skVES7

アドレスの検証:
- 先頭文字: n
- 長さ: 34 文字
- チェックサム: VES7

===mywalletのP2PKHアドレス生成===
公開鍵 (hex): 02ac8c65108e5a81e4c565b57f59241b0b94bb19541f28323c3bec76f3011f1e02
公開鍵 (長さ): 33 バイト

P2PKHアドレス: mgDU2f8H8E64fp4gioHa9gFyLNgz7av8st

アドレスの検証:
- 先頭文字: m
- 長さ: 34 文字
- チェックサム: v8st


#### Fund the 'sender' with 2.001 btc (0.001 btc is for the next tx fee)

In [6]:
txid_to_spend = node.sendtoaddress(sender_p2pkh_addr, 2.001)
print(txid_to_spend)

3f84aeff93e28667e84249690faa77af6d6cb16d01abadbf231f0ca31cbd2743


We can view the transaction using the bitcoin-cli commands `getrawtransaction` and `decoderawtransaction` as follows:

In [7]:
raw_tx = node.getrawtransaction(txid_to_spend)
decoded = node.decoderawtransaction(raw_tx)
print(raw_tx + "\n")
print(json.dumps(decoded, indent=2, default=str))

02000000000101e913685d7005b08d6ce1f4bd69c23d22c92cc6deed86c820f87b4a95bb96e2ac0000000000fdffffff02e49d181e010000001976a914b4174ecd0bfabbddab95a2fc5f809c94d4c15d2788aca048ed0b000000001976a914fc7250a211deddc70ee5a2738de5f07817351cef88ac0247304402200b29f711220a0d84c617dddb0dd38c9294c1f0b985c607f07966625a3103f3ab0220409c173d89d53b78e8ace3de1971317afe9ad93a21f3a2b3f7eeb7d03ad34c650121028f15290696e376494957267170456f2d654d69000fbdba99d18b8b4baf77fa5865000000

{
  "txid": "3f84aeff93e28667e84249690faa77af6d6cb16d01abadbf231f0ca31cbd2743",
  "hash": "b68037319c1c8340c82b6eedbeb606dc83ab99c8493ee3fe89ff50f295f00c29",
  "version": 2,
  "size": 228,
  "vsize": 147,
  "weight": 585,
  "locktime": 101,
  "vin": [
    {
      "txid": "ace296bb954a7bf820c886eddec62cc9223dc269bdf4e16c8db005705d6813e9",
      "vout": 0,
      "scriptSig": {
        "asm": "",
        "hex": ""
      },
      "txinwitness": [
        "304402200b29f711220a0d84c617dddb0dd38c9294c1f0b985c607f07966625a3103f3ab0220409c173d89

#### Find which output index the btc was sent to
Since we only sent 2.001 btc of the coinbase transaction (50 btc) to our address, bitcoind creates a change output to send the rest of the btc. By looking at the outputs we can see which is the change output and which was sent to our address. To do this in python we can do the following:

In [8]:
if decoded["vout"][0]["scriptPubKey"]["address"] == sender_p2pkh_addr:
    index_to_spend = 0
elif decoded["vout"][1]["scriptPubKey"]["address"] == sender_p2pkh_addr:
    index_to_spend = 1
else:
    raise Exception("couldn't find output")
print("index to spend from: " + str(index_to_spend))

index to spend from: 1


#### Mine a block so that the funding tx gets confirmed

In [9]:
wallet = node.get_wallet_rpc('mywallet')
address = wallet.getnewaddress()
node.generatetoaddress(nblocks=1, address=address, called_by_framework=True)

['77e71fdf6aec6209a3637912bbe6145c5b0e3508bda053706722b233b73db5ff']

## Spending a p2pkh UTXO

Now that we have some funds locked up in a p2pkh utxo, we can create a transaction spending from it. Let's say we want to send 1.5 btc to the address `mkxwE7XtVYJKepoD2hbHnDjftuMQ1k6deE`.

### Decoding a base58 address

The first thing we need to do is decode the address. This lets us:  
1 - validate the checksum to know the address was transmitted without error  
2 - make sure we are sending btc on the correct network (testnet/mainnet)  
3 - know what to put in the scriptPubkey

For more on addresses, refer back to the '[Addresses](https://github.com/DariusParvin/bitcoin-tx-tutorial/blob/main/appendix/addresses.ipynb)' chapter.

In [10]:
def decode_base58(address):
    """Base58Checkデコードを実行する関数"""
    import base58
    import hashlib
    
    # Base58デコード
    decoded = base58.b58decode(address)
    
    # バージョンバイトとチェックサムを分離
    version = decoded[0]
    checksum = decoded[-4:]
    payload = decoded[1:-4]
    
    # チェックサムの検証
    checksum_verify = hashlib.sha256(hashlib.sha256(decoded[:-4]).digest()).digest()[:4]
    if checksum != checksum_verify:
        raise ValueError("Invalid checksum")
    
    return {
        'version': version,
        'payload': payload,
        'checksum': checksum
    }

# テスト用アドレス
receiver_address = 'mkxwE7XtVYJKepoD2hbHnDjftuMQ1k6deE'

try:
    # アドレスをデコード
    decoded = decode_base58(receiver_address)
    pubkey_hash = decoded['payload']
    
    # 結果を表示
    print(f"アドレス: {receiver_address}")
    print("=" * 50)
    print(f"バージョンバイト: 0x{decoded['version']:02x} ({decoded['version']})")
    print(f"ペイロード (hex): {pubkey_hash.hex()}")
    print(f"ペイロード: {pubkey_hash}")
    print(f"チェックサム: {decoded['checksum'].hex()}")
    
    # ネットワークの判定
    if decoded['version'] == 0x6f:
        print("\nネットワーク: Testnet/Regtest")
    elif decoded['version'] == 0x00:
        print("\nネットワーク: Bitcoin Mainnet")
    else:
        print(f"\n不明なネットワーク (バージョン: 0x{decoded['version']:02x})")
    
    # アドレスタイプの判定
    if len(decoded['payload']) == 20:  # 20 bytes = 160 bits
        print("アドレスタイプ: P2PKH (Pay-to-Public-Key-Hash)")
        print(f"pubkey_hash (hex): {pubkey_hash.hex()}")
        print(f"pubkey_hash: {pubkey_hash}")
    elif len(decoded['payload']) == 32:  # 32 bytes = 256 bits
        print("アドレスタイプ: P2SH (Pay-to-Script-Hash)")
    else:
        print(f"不明なアドレスタイプ (ペイロード長: {len(decoded['payload'])}バイト)")
    
except Exception as e:
    print(f"エラーが発生しました: {e}")

アドレス: mkxwE7XtVYJKepoD2hbHnDjftuMQ1k6deE
バージョンバイト: 0x6f (111)
ペイロード (hex): 3bc28d6d92d9073fb5e3adf481795eaf446bceed
ペイロード: b';\xc2\x8dm\x92\xd9\x07?\xb5\xe3\xad\xf4\x81y^\xafDk\xce\xed'
チェックサム: ee2161b7

ネットワーク: Testnet/Regtest
アドレスタイプ: P2PKH (Pay-to-Public-Key-Hash)
pubkey_hash (hex): 3bc28d6d92d9073fb5e3adf481795eaf446bceed
pubkey_hash: b';\xc2\x8dm\x92\xd9\x07?\xb5\xe3\xad\xf4\x81y^\xafDk\xce\xed'


The first byte , in our case `6f`, tells us that this address corresponds to a p2pkh output for testnet. For more on decoding addresses, refer back to the 'Addresses' chapter.

Now we can create the receiver's output scriptPubkey:

In [11]:
receiver_spk = bytes.fromhex("76a914") + pubkey_hash + bytes.fromhex("88ac")
print(pubkey_hash)
print(receiver_spk)

b';\xc2\x8dm\x92\xd9\x07?\xb5\xe3\xad\xf4\x81y^\xafDk\xce\xed'
b'v\xa9\x14;\xc2\x8dm\x92\xd9\x07?\xb5\xe3\xad\xf4\x81y^\xafDk\xce\xed\x88\xac'


### Create an unsigned p2pkh transaction

The first thing we'll do is define the inputs and outputs of our transaction.

In [12]:
# Note we have already defined a few variables we need to create our transaction:
# The input utxo txid and index: `txid_to_spend` and `index_to_spend`
# The input private key and public key: `sender_privkey` and `sender_pubkey`

# Set our outputs
# Create a new pubkey to use as a change output.
change_privkey = bytes.fromhex("2222222222222222222222222222222222222222222222222222222222222222")
change_pubkey = privkey_to_pubkey(change_privkey)

# Determine our output scriptPubkeys and amounts (in satoshis)
output1_value_sat = int(float("1.5") * 100000000)
output1_spk = receiver_spk
output2_value_sat = int(float("0.5") * 100000000)
output2_spk = bytes.fromhex("76a914") + hash160(change_pubkey) + bytes.fromhex("88ac")

Now that we've defined everything we need, we can fill in the fields we need to create our unsigned transaction. What makes a transaction 'unsigned' is that the input's scriptSig, the field where the signature goes, is empty. This first step is necessary as the signature will cover the whole transaction (using SIGHASH_ALL). In a later chapter we will cover other sighash types and how they are signed.

In [13]:
# VERSION
# version '2' indicates that we may use relative timelocks (BIP68)
version = bytes.fromhex("0200 0000")

# INPUTS
# We have just 1 input
input_count = bytes.fromhex("01")

# Convert txid and index to bytes (little endian)
txid = (bytes.fromhex(txid_to_spend))[::-1]
index = index_to_spend.to_bytes(4, byteorder="little", signed=False)

# For the unsigned transaction we use an empty scriptSig
scriptsig = bytes.fromhex("")

# use 0xffffffff unless you are using OP_CHECKSEQUENCEVERIFY, locktime, or rbf
sequence = bytes.fromhex("ffff ffff")

inputs = (
    txid
    + index
    + varint_len(scriptsig)
    + scriptsig
    + sequence
)

# OUTPUTS
# 0x02 for out two outputs
output_count = bytes.fromhex("02")

# OUTPUT 1 
output1_value = output1_value_sat.to_bytes(8, byteorder="little", signed=True)
# 'output1_spk' already defined at the start of the script

# OUTPUT 2
output2_value = output2_value_sat.to_bytes(8, byteorder="little", signed=True)
# 'output2_spk' already defined at the start of the script

outputs = (
    output1_value
    + varint_len(output1_spk)
    + output1_spk
    + output2_value
    + varint_len(output2_spk)
    + output2_spk
)

# LOCKTIME
locktime = bytes.fromhex("0000 0000")

unsigned_tx = (
    version
    + input_count
    + inputs
    + output_count
    + outputs
    + locktime
)
print("unsigned_tx: ", unsigned_tx.hex())

unsigned_tx:  02000000014327bd1ca30c1f23bfadab016db16c6daf77aa0f694942e86786e293ffae843f0100000000ffffffff0280d1f008000000001976a9143bc28d6d92d9073fb5e3adf481795eaf446bceed88ac80f0fa02000000001976a914531260aa2a199e228c537dfa42c82bea2c7c1f4d88ac00000000


We can decode this raw transaction to inspect it and see that it has all the information we need apart from the scriptSig.

In [14]:
decoded = node.decoderawtransaction(unsigned_tx.hex())
print(json.dumps(decoded, indent=2, default=str))

{
  "txid": "e9872a56e77529af205cde106505771bc78d9c3a46ca82fcf1df7fe9e2e10efb",
  "hash": "e9872a56e77529af205cde106505771bc78d9c3a46ca82fcf1df7fe9e2e10efb",
  "version": 2,
  "size": 119,
  "vsize": 119,
  "weight": 476,
  "locktime": 0,
  "vin": [
    {
      "txid": "3f84aeff93e28667e84249690faa77af6d6cb16d01abadbf231f0ca31cbd2743",
      "vout": 1,
      "scriptSig": {
        "asm": "",
        "hex": ""
      },
      "sequence": 4294967295
    }
  ],
  "vout": [
    {
      "value": "1.50000000",
      "n": 0,
      "scriptPubKey": {
        "asm": "OP_DUP OP_HASH160 3bc28d6d92d9073fb5e3adf481795eaf446bceed OP_EQUALVERIFY OP_CHECKSIG",
        "desc": "addr(mkxwE7XtVYJKepoD2hbHnDjftuMQ1k6deE)#xlnzfr97",
        "hex": "76a9143bc28d6d92d9073fb5e3adf481795eaf446bceed88ac",
        "address": "mkxwE7XtVYJKepoD2hbHnDjftuMQ1k6deE",
        "type": "pubkeyhash"
      }
    },
    {
      "value": "0.50000000",
      "n": 1,
      "scriptPubKey": {
        "asm": "OP_DUP OP_HASH160 531

Before we can sign this transaction there is one final step we need to do. We need to replace the empty scriptSig with the scriptPubkey of the input we are signing over. If we had multiple inputs, we would need to do this step for each input. We will cover signing transactions with multiple inputs in a later chapter.

Since we are spending from a p2pkh utxo, we will create the scriptPubkey in the same way as we did for the outputs, but using the sender's pubkey:

In [15]:
pk_hash = hash160(sender_pubkey)
input_spk = bytes.fromhex("76a914" + pk_hash.hex() + "88ac")

inputs = (
    txid
    + index
    + varint_len(input_spk)
    + input_spk # replace the empty scriptSig with the input scriptPubkey
    + sequence
)

# tx hex to sign
tx_to_sign = (
    version
    + input_count
    + inputs
    + output_count
    + outputs
    + locktime
)

Now we are ready to hash this transaction and produce an ecdsa signature on it. 

Before hashing the transaction with hash256, we append the sighash flag. In this example we'll use the most commonly used SIGHASH_ALL flag, meaning the signature guarantees the input will only be used in a transaction with these exact inputs and outputs.

Note that when we append the sighash flag to the transaction, we use 4 bytes, however when we append the sighash flag to the end of the signature itself we only use 1 byte.

In [16]:
# Append the sighash flag to the transaction
sighash_flag = bytes.fromhex("0100 0000") # SIGHASH_ALL
sighash_preimage = tx_to_sign + sighash_flag

# Create sigHash to be signed
sighash = hash256(sighash_preimage)

# Sign the sigHash with the input private key
signing_key = ecdsa.SigningKey.from_string(sender_privkey, curve=ecdsa.SECP256k1) 
signature = signing_key.sign_digest(sighash, sigencode=ecdsa.util.sigencode_der_canonize)

# Append SIGHASH_ALL to the signature
signature = signature + bytes.fromhex("01")

# Signature
sig_script_signed = (
    pushbytes(signature)
    + pushbytes(sender_pubkey)
)

# tx_in with our new sigScript containing the signature we just created
inputs_signed = (
    txid
    + index
    + varint_len(sig_script_signed)
    + sig_script_signed
    + sequence
)

# the final signed transaction
signed_tx = (
    version
    + input_count
    + inputs_signed
    + output_count
    + outputs
    + locktime
)

print("signed transaction: ",signed_tx.hex())

signed transaction:  02000000014327bd1ca30c1f23bfadab016db16c6daf77aa0f694942e86786e293ffae843f010000006a4730440220353c4d137b321534dca0450a0bd80aec44c786e56fbf77e4711afd10e60f90840220418c99387fd12f23ca70d983243be9f40a92ba09e84de0f12c9db388d824d9270121034f355bdcb7cc0af728ef3cceb9615d90684bb5b2ca5f859ab0f0b704075871aaffffffff0280d1f008000000001976a9143bc28d6d92d9073fb5e3adf481795eaf446bceed88ac80f0fa02000000001976a914531260aa2a199e228c537dfa42c82bea2c7c1f4d88ac00000000


### Broadcast the transaction (on regtest mode)
If we get back a txid (32 byte hash), then it means the tx was successfully broadcast! If we just want to see if the transaction would have been accepted, but without broadcasting it, we can use the `testmempoolaccept` command (commented out).

In [17]:
new_tx_txid = node.sendrawtransaction(signed_tx.hex())
# new_tx_txid = node.testmempoolaccept([signed_tx.hex()])

print(new_tx_txid)

f8bfbf0d9de3c529f195b8b0a44d12b2f14a390ea470f06621dfba7edb8fdabc


We can decode the serialized transaction using ```decoderawtransction```. Notice that our output addresses match the change and receiver addresses from earlier.

In [18]:
print("receiver's p2pkh address: " + receiver_address)
change_p2pkh_addr = pk_to_p2pkh(change_pubkey, network = "regtest")
print("sender's change p2pkh address: " + change_p2pkh_addr)

receiver's p2pkh address: mkxwE7XtVYJKepoD2hbHnDjftuMQ1k6deE
sender's change p2pkh address: mo6CPsdW8EsnWdmSSCrQ6225VVDtpMBTug


In [19]:
decoded = node.decoderawtransaction(signed_tx.hex())
print(json.dumps(decoded, indent=2, default=str))

{
  "txid": "f8bfbf0d9de3c529f195b8b0a44d12b2f14a390ea470f06621dfba7edb8fdabc",
  "hash": "f8bfbf0d9de3c529f195b8b0a44d12b2f14a390ea470f06621dfba7edb8fdabc",
  "version": 2,
  "size": 225,
  "vsize": 225,
  "weight": 900,
  "locktime": 0,
  "vin": [
    {
      "txid": "3f84aeff93e28667e84249690faa77af6d6cb16d01abadbf231f0ca31cbd2743",
      "vout": 1,
      "scriptSig": {
        "asm": "30440220353c4d137b321534dca0450a0bd80aec44c786e56fbf77e4711afd10e60f90840220418c99387fd12f23ca70d983243be9f40a92ba09e84de0f12c9db388d824d927[ALL] 034f355bdcb7cc0af728ef3cceb9615d90684bb5b2ca5f859ab0f0b704075871aa",
        "hex": "4730440220353c4d137b321534dca0450a0bd80aec44c786e56fbf77e4711afd10e60f90840220418c99387fd12f23ca70d983243be9f40a92ba09e84de0f12c9db388d824d9270121034f355bdcb7cc0af728ef3cceb9615d90684bb5b2ca5f859ab0f0b704075871aa"
      },
      "sequence": 4294967295
    }
  ],
  "vout": [
    {
      "value": "1.50000000",
      "n": 0,
      "scriptPubKey": {
        "asm": "OP_DUP OP_HAS

## Quiz
1. How is the transaction ID calculated? 
2. Which field of a transaction contains the signature(s)? 
3. Bitcoin ECDSA signatures are vulnerable to third party malleation. As a result, it is possible for a third party to alter the transaction ID. If Alice is broadcasting a transaction to Bob, how might a third party manage to change the transaction ID using only publicly available information?
4. What problems or limitations does this transaction malleability cause?

 ## Answers
1. It is the double SHA256 (HASH256) of the raw transaction. Note that the output is displayed in little endian. See '[The First Bitcoin Transaction (Pay to Pubkey)](https://github.com/DariusParvin/bitcoin-tx-tutorial/blob/main/chapter1-legacy/first-btc-tx.ipynb)' for an example.
2. The scriptSig in the input field contains the signature(s).
3. A third party could observe the transaction in the mempool, change the signature, then broadcast the transaction with the altered signature and transaction ID.
4. Two issues (non exhaustive):
  - It makes it impossible to create a chain of offline transactions as the inputs of a child may change by the time it is confirmed. This is an issue for many layer 2 protocols such as lightning payment channels.
  - If Alice or Bob are searching for the transaction using the transaction ID, they may never find it. 

## Exercise
Now that we have the change output with a known private key `change_privkey`, create a transaction that spends from it to two outputs. The first output should to send 0.1 btc to the address `mgiS1dSDrPunE7GvXmoS4xEfmdwWsStZc7`. The second address should send the rest to a change output with the address `mz8AXDhDMhvLs7kxwfQxvcH5GoVH6AdARZ`. Set the miner fee to 0.001 btc.

The first step will be to decode the output addresses we want to send btc to and convert them into their scriptPubkeys.

## Solution

In [20]:
# Solution - Converting the addresses to scriptPubKeys
receiver_address = 'mgiS1dSDrPunE7GvXmoS4xEfmdwWsStZc7'
receiver_address_decoded = decode_base58(receiver_address)
receiver_pubkey_hash = receiver_address_decoded[1:-4] 
output1_spk = bytes.fromhex("76a914") + receiver_pubkey_hash + bytes.fromhex("88ac")

change_address = 'mz8AXDhDMhvLs7kxwfQxvcH5GoVH6AdARZ'
change_address_decoded = decode_base58(change_address)
change_pubkey_hash = change_address_decoded[1:-4] 
output2_spk = bytes.fromhex("76a914") + change_pubkey_hash + bytes.fromhex("88ac")

Now let's create an unsigned transaction. For now we will keep the scriptSig empty (`0x00`)

In [21]:
# Solution - creating an unsigned transaction
version = bytes.fromhex("0200 0000")

# INPUTS
input_count = bytes.fromhex("01")

# INPUT 1
txid_to_spend = new_tx_txid
txid = (bytes.fromhex(txid_to_spend))[::-1]
index_to_spend = 1
index = index_to_spend.to_bytes(4, byteorder="little", signed=False)
scriptsig = bytes.fromhex("")
sequence = bytes.fromhex("ffff ffff")

inputs = (
    txid
    + index
    + varint_len(scriptsig)
    + scriptsig
    + sequence
)

# OUTPUTS
output_count = bytes.fromhex("02")

# OUTPUT 1 
output1_value_sat = int(float("0.1") * 100000000)
output1_value = output1_value_sat.to_bytes(8, byteorder="little", signed=True)

# OUTPUT 2
output2_value_sat = int(float("0.399") * 100000000) # 0.5 - 0.1 - 0.001 for the miner fee
output2_value = output2_value_sat.to_bytes(8, byteorder="little", signed=True)

outputs = (
    output1_value
    + varint_len(output1_spk)
    + output1_spk
    + output2_value
    + varint_len(output2_spk)
    + output2_spk
)

# LOCKTIME
locktime = bytes.fromhex("0000 0000")

unsigned_tx = (
    version
    + input_count
    + inputs
    + output_count
    + outputs
    + locktime
)

# print(unsigned_tx.hex())

Replace the scriptSig with the input's scriptPubKey. You can either code this yourself or take it directly from the decoded transaction. The `"hex"` field of the output we are spending from can be used directly.

In [22]:
# Solution - replace the scriptPubKey for creating the sigHash
input_spk = bytes.fromhex("76a914531260aa2a199e228c537dfa42c82bea2c7c1f4d88ac")

In [23]:
# Solution - creating the sigHash and signing the transaction
inputs = (
    txid
    + index
    + varint_len(input_spk)
    + input_spk
    + sequence
)

tx_to_sign = (
    version
    + input_count
    + inputs
    + output_count
    + outputs
    + locktime
)


# Append the sighash flag to the transaction
sighash_flag = bytes.fromhex("0100 0000") # SIGHASH_ALL
sighash_preimage = tx_to_sign + sighash_flag

# Create sigHash to be signed
sighash = hash256(sighash_preimage)

# Sign the sigHash with the input private key
signing_key = ecdsa.SigningKey.from_string(change_privkey, curve=ecdsa.SECP256k1) 
signature = signing_key.sign_digest(sighash, sigencode=ecdsa.util.sigencode_der_canonize)

# Append SIGHASH_ALL to the signature
signature = signature + bytes.fromhex("01")

# Signature
sig_script_signed = (
    pushbytes(signature)
    + pushbytes(change_pubkey)
)

# tx_in with our new sigScript containing the signature we just created
inputs_signed = (
    txid
    + index
    + varint_len(sig_script_signed)
    + sig_script_signed
    + sequence
)

# the final signed transaction
signed_tx = (
    version
    + input_count
    + inputs_signed
    + output_count
    + outputs
    + locktime
)

# print("signed transaction: ",signed_tx.hex())

In [24]:
# Try broadcasting the transaction to see if it works!
node.sendrawtransaction(signed_tx.hex())

'c4ed94e06172f08e7508e419e2dfa1060b83232bdc47245d408981ea3723c96d'

In [4]:
# stop bitcoin core
test.shutdown()

2025-12-02T17:44:41.820296Z TestFramework (INFO): Stopping nodes
2025-12-02T17:44:41.927120Z TestFramework (INFO): Cleaning up /var/folders/yk/mx8jn78j6xq0w5l4z9d8hbvw0000gn/T/bitcoin_func_test_1s1nxa5r on exit
2025-12-02T17:44:41.928481Z TestFramework (INFO): Tests successful
